# Toy example for Radix-based Benders

In [1]:
SCALE_CROWDING= False
PREVENT_ZERO  = False

In [2]:
%load_ext line_profiler

In [3]:
from gurobipy import *

import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams['svg.fonttype'] = 'none'
pd.set_option('display.max_colwidth', -1)
%matplotlib inline

from dynamicme.decomposition import Decomposer
from dynamicme.callback_gurobi import cb_benders
from dynamicme.optimize import Optimizer, StackOptimizer
from dynamicme.optimize import Constraint, Variable

from cobra.io import load_json_model
from cobra import Metabolite, Reaction
from six import iteritems

import numpy as np
import cobra

(<type 'exceptions.ImportError'>, ImportError('No module named cplex',), <traceback object at 0x7f0568a89ef0>)


### Try optimizing using radix for one condition first

In [4]:
#----------------------------------------
# Starting from basal model
ijomc = load_json_model('/home/laurence/ME/models/BiGG_M/json/e_coli_core.json')
keff0 = 1./65/3600
#crowding_bound = 0.0003
crowding_bound0 = 0.001
crowding_bound  = crowding_bound0
not_crowded = ['ATPM']
rxns_c = [r for r in ijomc.reactions if all([m.compartment=='c' for m in r.metabolites.keys()]) and 'BIOMASS' not in r.id and r.id not in not_crowded]
crowding_dict = {rxn:keff0 for rxn in rxns_c}
#----------------------------------------

# Temporarily add crowding constraint for the duality gap constraint
crowding = Constraint('crowding')
crowding._bound = crowding_bound
crowding._constraint_sense = 'L'
for rxn,keff in iteritems(crowding_dict):
    rxn.add_metabolites({crowding:keff})

In [5]:
ijomc.optimize()
mu_crowd0 = ijomc.reactions.BIOMASS_Ecoli_core_w_GAM.x
print(mu_crowd0)

0.873921506968


In [6]:
opt = Optimizer(ijomc)
gap = opt.add_duality_gap_constraint(INF=1e3)

In [7]:
gap.optimize(solver='gurobi')

<Solution 1.75 at 0x7f05ac1f6d10>

In [8]:
import numpy as np

radix = 2.
powers = np.arange(-3,4)
print(powers)
digits_per_power = radix
pwr_max = max(powers)
digits = list(set(np.linspace(1, radix-1, digits_per_power)))
print(digits)

# Discretize crowding coefficients into radix
crowding_p = gap.metabolites.crowding

var_cons_dict = {}
for rxn_p in crowding_p.reactions:
    # Get the coefficient in the dual
    var_d = gap.reactions.wa_crowding
    cons_ds = [m for m in var_d.metabolites.keys() if rxn_p.id==m.id]
    a0 = rxn_p.metabolites[crowding_p]    
    var_cons_dict[rxn_p.id] = [(rxn_p, crowding_p, a0)] + [(var_d, cons_d, a0) for cons_d in cons_ds]

[-3 -2 -1  0  1  2  3]
[1.0]


/usr/lib/python2.7/site-packages/ipykernel/__main__.py:8 DeprecationWarning: object of type <type 'float'> cannot be safely interpreted as an integer.


In [9]:
#%lprun -f opt.to_radix opt.to_radix(gap, var_cons_dict, radix, powers, digits_per_power)

In [10]:
%%time
opt.to_radix(gap, var_cons_dict, radix, powers, digits=digits, prevent_zero=PREVENT_ZERO)

CPU times: user 126 ms, sys: 10.4 ms, total: 136 ms
Wall time: 123 ms


[1.0]

In [11]:
### Try scaling
if SCALE_CROWDING:
    crowding = gap.metabolites.crowding
    bound0  = crowding._bound
    f_scale = crowding._bound
    crowding._bound = bound0 / f_scale
    for rxn in crowding.reactions:
        rxn._metabolites[crowding] = rxn._metabolites[crowding]/f_scale

In [12]:
for rxn in gap.reactions:
    rxn.objective_coefficient = 0.
for rxn in gap.reactions.query('binary'):
    rxn.objective_coefficient = 1.

In [13]:
N_CONDS = 1

df_meas = pd.read_csv('/home/laurence/ME/data/dynamicME/beg/growth_meas.csv')
ex_rxns = [r.id for r in ijomc.reactions.query('EX_')]
df_meas = df_meas[ df_meas.ex_rxn.isin(ex_rxns)]
conds = df_meas.substrate.unique()
df_conds = pd.DataFrame([{'cond':r['substrate'], 'rxn':ex_rxn, 'lb':-10 if r['ex_rxn']==ex_rxn else 0, 'ub':1000., 'obj':0.} for i,r in df_meas.iterrows() for ex_rxn in ex_rxns])
df_conds = df_conds[ df_conds.cond.isin(conds[0:N_CONDS])]

In [16]:
mu_meas = 0.74

F_TOL = 0.02
rxn_mu = gap.reactions.BIOMASS_Ecoli_core_w_GAM
rxn_mu.lower_bound = mu_meas*(1-F_TOL)
rxn_mu.upper_bound = mu_meas*(1+F_TOL)
print('Initial mu_crowd=%g. Fitting within %g%% of measured: %g <= mu <= %g' % (mu_crowd0, 100*F_TOL, rxn_mu.lower_bound, rxn_mu.upper_bound))

    
from cobra.solvers import gurobi_solver
from gurobipy import *

milp = gurobi_solver.create_problem(gap)
milp.ModelSense = GRB.MINIMIZE

Initial mu_crowd=0.873922. Fitting within 2% of measured: 0.7252 <= mu <= 0.7548


In [17]:
milp.Params.FeasibilityTol = 1e-9
milp.Params.OptimalityTol = 1e-9
milp.Params.IntFeasTol = 1e-9
milp.Params.OutputFlag = 1
milp.optimize()

Changed value of parameter OutputFlag to 1
   Prev: 0  Min: 0  Max: 1  Default: 1
Optimize a model with 3193 rows, 1366 columns and 8292 nonzeros
Variable types: 1030 continuous, 336 integer (0 binary)
Coefficient statistics:
  Matrix range     [5e-07, 1e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [7e-01, 1e+03]
  RHS range        [1e-03, 1e+03]
Presolve removed 1173 rows and 183 columns
Presolve time: 0.01s
Presolved: 2020 rows, 1183 columns, 5982 nonzeros
Variable types: 854 continuous, 329 integer (329 binary)

Root relaxation: objective 5.885411e-01, 1038 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.58854    0   10          -    0.58854      -     -    0s
     0     0    0.58854    0   10          -    0.58854      -     -    0s
     0     0    0.76364    0   10          -    0.76364      -     -    0s
     0     2    0.

In [18]:
#gap.optimize('minimize')
rxn_mu =gap.reactions.BIOMASS_Ecoli_core_w_GAM 
#muopt = rxn_mu
sol = gurobi_solver.format_solution(milp, gap)
muopt = sol.x_dict[rxn_mu.id]
yopt = [sol.x_dict[rxn.id] for rxn in gap.reactions.query('binary_')]
sum(yopt)
print('Initial mu_crowd=%g. Fitted within %g%%: %g <= %g <= %g' % (mu_crowd0, 100*F_TOL, rxn_mu.lower_bound, muopt, rxn_mu.upper_bound))
print('Number of non-zero binaries: %g' % sum(yopt))

Initial mu_crowd=0.873922. Fitted within 2%: 0.7252 <= 0.733706 <= 0.7548
Number of non-zero binaries: 3


In [19]:
# Fitted parameters
kfit_dict = {}
for group_id, var_dict in iteritems(var_cons_dict):
    var = var_dict[0]
    cons = var_dict[1]
    a0  = var_dict[0][2]
    kfit = 0.
    for l,pwr in enumerate(powers):
        for k,digit in enumerate(digits):            
            yid = 'binary_%s%s%s' % (group_id,k,l)
            y   = sol.x_dict[yid]
            if abs(y)>1e-10:
                print yid, y
            kfit += y*a0*radix**pwr*digit
    kfit_dict[group_id] = kfit

binary_PGL06 1.0
binary_GAPD06 1.0
binary_ENO06 1.0


In [20]:
kfit_dict1 = dict(kfit_dict)

In [21]:
'Binaries per keff:', len(gap.reactions.query('binary')) / len(kfit_dict)

('Binaries per keff:', 7)

In [22]:
len(var_cons_dict)

48

In [23]:
[(k,v) for k,v in iteritems(kfit_dict) if abs(v)>1e-10]

[(u'PGL', 3.418803418803419e-05),
 (u'GAPD', 3.418803418803419e-05),
 (u'ENO', 3.418803418803419e-05)]

### Plug back in to be sure

In [24]:
#----------------------------------------
# Starting from basal model
ijofit = load_json_model('/home/laurence/ME/models/BiGG_M/json/e_coli_core.json')
crowding = Constraint('crowding')
crowding._bound = crowding_bound0
crowding._constraint_sense = 'L'
for rid,kfit in iteritems(kfit_dict):
    rxn = ijofit.reactions.get_by_id(rid)
    rxn.add_metabolites({crowding:kfit})

ijofit.optimize()
ijofit.reactions.BIOMASS_Ecoli_core_w_GAM.x

'Unoptimized: %g. Fitted: %g' % (mu_crowd0, ijofit.reactions.BIOMASS_Ecoli_core_w_GAM.x)

'Unoptimized: 0.873922. Fitted: 0.733706'

# Try using Benders

In [25]:
decomp = Decomposer(gap, objective_sense='minimize')

In [26]:
master,sub = decomp.benders_decomp()

master.Params.presolve = 0
sub.Params.presolve = 0

master.Params.FeasibilityTol = 1e-9
master.Params.OptimalityTol = 1e-9

sub.Params.Method = 0
sub.Params.OutputFlag = 0

sub.Params.FeasibilityTol = 1e-9
sub.Params.OptimalityTol = 1e-9

master._verbosity = 0
master._precision_sub = 'gurobi' #'dq'
master.Params.OutputFlag=0

Changed value of parameter Presolve to 0
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter LazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter IntFeasTol to 1e-09
   Prev: 1e-05  Min: 1e-09  Max: 0.1  Default: 1e-05
Changed value of parameter InfUnbdInfo to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Parameter presolve unchanged
   Value: 0  Min: -1  Max: 2  Default: -1
Changed value of parameter presolve to 0
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter FeasibilityTol to 1e-09
   Prev: 1e-06  Min: 1e-09  Max: 0.01  Default: 1e-06
Changed value of parameter OptimalityTol to 1e-09
   Prev: 1e-06  Min: 1e-09  Max: 0.01  Default: 1e-06
Changed value of parameter Method to 0
   Prev: -1  Min: -1  Max: 5  Default: -1


In [27]:
# ybad = np.array([0. for y in decomp._ys])
# psub = decomp.make_sub_primal(ybad)
# psub.model.Params.InfUnbdInfo=1
# psub.model.optimize()

In [28]:
# ### DEBUG ****************
# # Test an unbounded subproblem using a yopt that is known to be infeasible for primal
# ybad = np.array([0. for y in decomp._ys])
# decomp.update_subobj(ybad)
# sub.model.Params.ModelSense = GRB.MAXIMIZE
# sub.optimize()
# print('Status: %s' % sub.model.Status)
# print('Status is unbounded: %s' % (sub.model.Status==GRB.UNBOUNDED))
# print('Obj val: %s' %sub.ObjVal)

In [29]:
WARM_START_BENDERS = False
# **** DEBUG: warm-start with known solution ****
if WARM_START_BENDERS:
    for yj in master.getVars():
        if 'binary' in yj.VarName:
            yj.Start = sol.x_dict[yj.VarName]
            if sol.x_dict[yj.VarName] > 1e-10:
                print((yj, sol.x_dict[yj.VarName]))

In [30]:
master._verbosity=0
master.Params.OutputFlag = 1
%time master.optimize(cb_benders)

Changed value of parameter OutputFlag to 1
   Prev: 0  Min: 0  Max: 1  Default: 1
Optimize a model with 337 rows, 337 columns and 673 nonzeros
Variable types: 1 continuous, 336 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+03]
  RHS range        [1e+00, 1e+00]
Variable types: 1 continuous, 336 integer (336 binary)

Root relaxation: objective 1.000000e+00, 2 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    1.00000    0    2          -    1.00000      -     -    0s
     0     0    1.00000    0    2          -    1.00000      -     -    0s
     0     0    1.00000    0    2          -    1.00000      -     -    0s
     0     2    1.00000    0    2          -    1.00000      -     -    0s
   278   221    3.25346   68    4          -    1.42286      -   1.9

In [31]:
master.ObjVal

2.999999999999938

In [32]:
sub.ObjVal

-0.0

In [33]:
yoptB = np.array([y.X for y in decomp._ys])
sum(yoptB)

3.0

In [34]:
mu_id = 'BIOMASS_Ecoli_core_w_GAM'

cons_mu = sub.model.getConstrByName(str(mu_id))
cons_mu.Pi

0.7548

In [35]:
sol_master = {v.VarName:v.X for v in master.getVars()}

In [36]:
# Fitted parameters
kfit_dict = {}
for group_id, var_dict in iteritems(var_cons_dict):
    var = var_dict[0]
    cons = var_dict[1]
    a0  = var_dict[0][2]
    kfit = 0.
    for l,pwr in enumerate(powers):
        for k,digit in enumerate(digits):            
            yid = 'binary_%s%s%s' % (group_id,k,l)
            y   = sol_master[yid]
            if abs(y)>1e-10:
                print('%s. Value=%s. Power=%g. Digit=%g' % (yid, y, pwr, digit))
            kfit += y*a0*radix**pwr*digit
    kfit_dict[group_id] = kfit

binary_G6PDH2r06. Value=1.0. Power=3. Digit=1
binary_GAPD06. Value=1.0. Power=3. Digit=1
binary_ENO06. Value=1.0. Power=3. Digit=1


In [37]:
[(k,v) for k,v in iteritems(kfit_dict) if abs(v)>1e-10]

[(u'G6PDH2r', 3.418803418803419e-05),
 (u'GAPD', 3.418803418803419e-05),
 (u'ENO', 3.418803418803419e-05)]

In [38]:
#----------------------------------------
# Starting from basal model
ijofit = load_json_model('/home/laurence/ME/models/BiGG_M/json/e_coli_core.json')
crowding = Constraint('crowding')
crowding._bound = crowding_bound0
crowding._constraint_sense = 'L'
for rid,kfit in iteritems(kfit_dict):
    rxn = ijofit.reactions.get_by_id(rid)
    rxn.add_metabolites({crowding:kfit})

ijofit.optimize()
ijofit.reactions.BIOMASS_Ecoli_core_w_GAM.x

'Unoptimized: %g. Fitted: %g' % (mu_crowd0, ijofit.reactions.BIOMASS_Ecoli_core_w_GAM.x)

'Unoptimized: 0.873922. Fitted: 0.733706'